#### Vacuum Agent Return Path Planning

Help your vacuum agent go home to charge as efficiently as possible!

While the agent is in explore mode, it records every position it visits, then when it needs to go home, it computes the shortest path from its current position to home position [1,1] that goes only through positions it visited while exploring.  You are going to write a function
<pre>
agent_path_home(list_of_positions, start_position)  => path
</pre>
where the return path is a list of *headings* ("North", "South", "East", "West").   If the agent starts at the start_position -- the position where it stops exporing and starts going home -- and repeatedly moves in the headings specified by the path, it will end up at square [1,1].  The path returned must be the shortest possible path from the start position to [1,1].

You must use the search framework library for your solution.

Here is an example for a 6x6 grid
<img src="grid.GIF" alt="grid" style="width:200px;"/>

The full exploration path is 15 steps
```
(1,1), (1,2), (2,2), (2,1), 
(1,1), (2,1), (3,1), (3,2), 
(3,3), (3,4), (3,3), (4,3), 
(5,3), (5,2), (5,1), (6,1)
```
the shortest path from (6,1) back to (1,1) has 9 steps, and one such path is
```
(6,1), (5,1), (5,2), (5,3),
(4,3), (3,3), (3,2), (2,2), 
(1,2), (1,1)
```
The path from (6,1) back to (1,1) would be represented as
```
[North, East, East, North, North, West, North, North, West]
```
so calling your code would look like this:
<pre>
positions = [(1,1), (1,2), (2,2), (2,1), (1,1), (2,1), (3,1), 
             (3,2), (3,3), (3,4), (3,3), (4,3), (5,3), (5,2), (5,1), (6,1)]
starting_position = (6,1)
path = agent_path_home(positions, starting_position)
print(path)

['North', 'East', 'East', 'North', 'North', 'West', 'North', 'North', 'West']
</pre>

### Helper Functions
It's always good to experiment and test, so here are some functions to help you out.

In [1]:
#  This function generates a randomly generated position list that 
#  can be used as input to agent_path_home.  Inputs are size (for example 10 for a 10x10 grid), and a
#  path length -- this is the number of steps the agent will explore, not the length
#  of the shortest path home

from random import randint

def problem_generator(size, walk_length):
    
    def inbounds(position, size):
        return position[0] >= 1 and position[1] <= size and position[1] > 1 and position[1] <= size
    
    positions = [(1,1)]
    for _ in range(walk_length):
        newposition = None
        while True:
            r,c = positions[-1]
            x = randint(0,3)
            newposition = [(r+1, c), (r-1, c), (r, c-1), (r, c+1)][x]
            if inbounds(newposition, size):
                break
        positions.append(newposition)
    return (positions, positions[-1])
            

```
Here is an example with a big environment and lots of wandering.  Shortest path much better!

SIZE = 200
PATH_LENGTH = 400

path, start_position = problem_generator(SIZE, PATH_LENGTH)
shortest = agent_path_home(path,start_position)

print(f"Full path is length {PATH_LENGTH}, start position is {start_position}, shortest path is length {len(shortest)}")

Full path is length 400, start position is (4, 6), shortest path is length 8
```

Here's another helper function and a hint.

When you do your state-space search, chances are your state will just be 
the position the agent is in.  For example in the example above, your start
state will be (4,6) and the successor states will be all positions on the path
that are adjacent to (4,6) -- let's say those are (3,6) and (4,5).
But the "actions" in your state-space search are headings -- North, South, East, West
So you need a function that gives you an "action" aka "heading" given two adjacent positions.

```
print(heading_for_adjacency((4,6), (3,6)))
print(heading_for_adjacency((4,6), (4,5))
   North
   East
```


In [2]:
def heading_for_adjacency(p1, p2):
    if (p2[0] == p1[0]-1):
        return "North"
    elif (p2[0] == p1[0]+1):
        return "South"
    elif (p2[1] == p1[1]+1):
        return "East"
    elif (p2[1] == p1[1]-1):
        return "West"
    else: 
        raise(Exception(f"Not adjacent {p1} {p2}"))

In [3]:
##  And here is yet another helper function for you to help you debug
##  your search code.  It takes problem input (free positions and start position) as
##  input, and a solution as returned by your agent_path_home function, and verifies
##  that the solution is correct.

def check_solution(clear, initial, path):   
    def pos_in_heading(p,h):
        if (h == "North"):
            return (p[0]-1, p[1])
        elif (h == "South"):
            return (p[0]+1, p[1])
        elif (h == "West"):
            return (p[0], p[1]-1)
        elif (h == "East"):
            return (p[0], p[1]+1)
        else:
            raise Exception(f"Not a direction {h}")
    pos = initial
    for heading in path:
        np = pos_in_heading(pos, heading)
        if (not np in clear):
            return False
        pos = np
    return pos == (1,1)


For example, 
```
p, s = problem_generator(20, 20)
print(check_solution(p, s, agent_path_home(p, s)))

True
```

In [4]:
## Your code for the class definitions and function agent_path_home goes in this cell.   
## It solves the path problem using breadth-first search.  
## Line 6 is a hint!  You don't have to use it as is

def agent_path_home(path, start_position):
    return aStarSearch(VacuumProblem(path, start_position), BFSEvaluator())[0]

####  Heuristic and Empirical Evaluation

In this section you will develop a heuristic evaluation function -- h*(s) and compare path search using your heuristic with breadth-first search.  You will compare the two using the metric "number of nodes explored" -- the second element returned in the stats record returned by A Star Search.

Notice that since we are interested only in the stats, you will be calling A Star Search function directly, not
the agent_path_home function you developed in the previous section

In [10]:
## Fill this in.  Skeleton only, this does BFS

from searchClientInterface import Evaluator
def heuristicEstimator(state):
    return 0
def heuristicCoster(actions):
    return len(actions)
def heuristicEvaluator():
    return Evaluator(heuristicCoster, heuristicEstimator)

```
# Here's an example that generates a random problem, calls search
# using BFS and the heuristic, verifies that the two searches return
# paths of the same lengths, then prints out the stat records for both

from searchFramework import aStarSearch
clear, initial = problem_generator(1000, 300)
problem = VacuumProblem(clear, initial)
solnb, statsb = aStarSearch(problem, Evaluator(lambda a: len(a), lambda s: 0))
solnh, statsh = aStarSearch(problem, heuristicEvaluator())

print(len(solnb))
print(len(solnh))
print(statsb)
print(statsh)

48
48
(0.828125, 875, 719, 56)
(0.5, 626, 508, 297)
```

In [ ]:
# Here's one last example where I do experiments on 50x50 grids 
# with an exploration path of about 200.  I generate 1000 such problems and solutions, and compare
# number of nodes expanded by bfs versus heuristic

from statistics import stdev
from statistics import mean
from searchClientInterface import BFSEvaluator

diffs = []
for _ in range(1,100):
    clear, initial = problem_generator(50, 200 + randint(-10, 10))
    problem = VacuumProblem(clear, initial)
    solnb, statsb = aStarSearch(problem, BFSEvaluator())
    solnh, statsh = aStarSearch(problem, heuristicEvaluator())
    diffs.append(statsb[1] - statsh[1])

print(mean(diffs))
print(stdev(diffs))

355.1111111111111
158.09031833997446

#### Analysis

Here you will analyze your heuristic performance versus breadth-first search.
You want to answer these two questions:
1.  In terms of nodes expanded, does your heuristic significantly outperform BFS
2.  In terms of time elapsed, does your heuristic significantly outperform BFS
3.  In terms of time per node, does your heuristic significantly outperform BFS

Keep in mind the second two questions are interesting:  even if your heuristic does guide the search effectively toward a solution, if it takes too long to estimate a node's "quality,"  it still might not be worth using.  

---


<p style="color:red"> Your analysis in this cell and add cells below if you want </p>